In [ ]:
from Crypto.Cipher import AES
import os

KEY = None
FLAG = None


# @chal.route('/symmetry/encrypt/<plaintext>/<iv>/')
def encrypt(plaintext, iv):
    plaintext = bytes.fromhex(plaintext)
    iv = bytes.fromhex(iv)
    if len(iv) != 16:
        return {"error": "IV length must be 16"}

    cipher = AES.new(KEY, AES.MODE_OFB, iv)
    encrypted = cipher.encrypt(plaintext)
    ciphertext = encrypted.hex()

    return {"ciphertext": ciphertext}


# @chal.route('/symmetry/encrypt_flag/')
def encrypt_flag():
    iv = os.urandom(16)

    cipher = AES.new(KEY, AES.MODE_OFB, iv)
    encrypted = cipher.encrypt(FLAG.encode())
    ciphertext = iv.hex() + encrypted.hex()

    return {"ciphertext": ciphertext}

In [3]:
from Crypto.Cipher import AES
import os
import requests

ENCODE_URL = "https://aes.cryptohack.org/symmetry/encrypt_flag/"
DECODE_URL = "https://aes.cryptohack.org/symmetry/encrypt/"

def get_respone(url):
    respone = requests.get(url).json()
    keys = respone.keys()
    for k in ("ciphertext", "plaintext", "cookie", "flag"):
        if k in keys:
            return respone[k]
        
def hex_XOR(a, b):
    assert len(a) == len(b)
    return bytes(x ^ y for x, y in zip(bytes.fromhex(a), bytes.fromhex(b))).hex()

# Get iv:
ciphertext = get_respone(ENCODE_URL)

iv = ciphertext[:32]

pesudo_cipher = (f"{chr(0)}" * (len(ciphertext) // 2 - 16)).encode('utf-8').hex()

iv_cipher = get_respone(DECODE_URL + pesudo_cipher + "/" + iv + "/")

FLAG = bytes.fromhex(hex_XOR(ciphertext[32:], iv_cipher)).decode('utf-8')
print(FLAG)


crypto{0fb_15_5ymm37r1c4l_!!!11!}
